In [60]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import bs4

In [8]:
headers = {
   'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
}

In [16]:
base_url = 'https://www.transfermarkt.co.uk/'
region_suffix = 'wettbewerbe/'
region_url = base_url + region_suffix
regions_list = ['europa', 'asien', 'amerika', 'afrika']

In [26]:
for r in regions_list:
    print(r)
    b = requests.get(region_url + r, headers=headers)
    break

europa


In [27]:
soup = BeautifulSoup(b.content, 'html5lib')

In [152]:
header = get_header(soup)

In [161]:
real_header = header[:6] + header[7:]

In [167]:
real_header

['Competition',
 'Country',
 'Clubs',
 'Player',
 'Avg. age',
 'Foreign players',
 'Total value']

In [175]:
def get_header(soup):
    table = soup.find(name='table', attrs={'class': 'items'})
    header = table.find('thead')
    header_cols = header.findAll('th')
    header_parsed = []
    for hc in header_cols:
        if type(hc.contents[0]) == bs4.element.NavigableString:
            value = hc.contents[0]
        else:
            value = hc.contents[0].contents[0]
        header_parsed.append(value)
    return header_parsed

def get_content_table(soup):
    table = soup.find(name='table', attrs={'class': 'items'})
    table_body = table.find(name='tbody')
    rows = table_body.findAll('tr', attrs={'class': ['odd', 'even']})
    t = []
    for r in rows:
        cols = r.findAll('td')
        t.append(get_row_values(cols))
    return t

def get_row_values(cols):
    print(cols)
    values = []
    for i, c in enumerate(cols):
        content = c.contents
        if len(content) == 0:
            continue
        value = c.contents[0]
        if type(value) is bs4.element.NavigableString:
            values.append(value)
        else:
            a_list = c.findAll('a')
            if len(a_list) == 0:
                values.append(c.find('img')['alt'])
            elif len(a_list) == 1:
                values.append(a_list[0].contents[0])
            else:
                values.append(a_list[1].contents[0])
    return values

header = ['Competition',
 'Country',
 'Clubs',
 'Player',
 'Avg. age',
 'Foreign players',
 'Total value']

def treat_raw_table(raw_table):
    raw_table = raw_table.drop([1,2], axis=1)
    raw_table['merged'] = raw_table.apply(lambda x: x[8] if x[9] is None else x[9], axis=1)
    raw_table = raw_table.drop([8,9], axis=1)
    raw_table.columns = header
    return raw_table

In [176]:
data = pd.DataFrame(get_content_table(soup))

[<td class="hauptlink"><table class="inline-table">
	<tbody><tr>
		<td>
			<a href="/premier-league/startseite/wettbewerb/Premier+League">
			<img alt="Premier League" class="" src="https://tmssl.akamaized.net//images/logo/tiny/gb1.png?lm=1521104656" title="Premier League"/>			</a>
		</td>
		<td>
            <a href="/premier-league/startseite/wettbewerb/GB1" title="Premier League">Premier League</a>
		</td>
	</tr>
</tbody></table></td>, <td>
			<a href="/premier-league/startseite/wettbewerb/Premier+League">
			<img alt="Premier League" class="" src="https://tmssl.akamaized.net//images/logo/tiny/gb1.png?lm=1521104656" title="Premier League"/>			</a>
		</td>, <td>
            <a href="/premier-league/startseite/wettbewerb/GB1" title="Premier League">Premier League</a>
		</td>, <td class="zentriert"><img alt="England" class="flaggenrahmen" src="https://tmssl.akamaized.net//images/flagge/tiny/189.png?lm=1520611569" title="England"/></td>, <td class="zentriert">20</td>, <td class="zentrier

In [172]:
treat_raw_table(data)

,Competition,Country,Clubs,Player,Avg. age,Foreign players,Total value
0,Premier League,England,20,519,"27,0","68,0 %",£7.21bn
1,LaLiga,Spain,20,472,"27,0","41,3 %",£4.56bn
2,Serie A,Italy,20,563,"26,4","57,4 %",£3.95bn
3,1.Bundesliga,Germany,18,521,"25,2","52,4 %",£3.41bn
4,Ligue 1,France,20,542,"25,4","48,9 %",£2.61bn
5,Liga NOS,Portugal,18,514,"26,0","62,6 %",£848.41m
6,Premier Liga,Russia,16,400,"27,3","37,0 %",£705.89m
7,Eredivisie,Netherlands,18,488,"24,1","38,3 %",£688.82m
8,Süper Lig,Turkey,18,496,"27,5","50,4 %",£599.72m
9,Jupiler Pro League,Belgium,16,472,"24,8","63,6 %",£482.11m
